In [1]:
import tkinter as tk
from tkinter import ttk, messagebox, scrolledtext
import threading
import queue
from collections import deque
import heapq
import time
from datetime import datetime
import matplotlib
matplotlib.use('TkAgg')  # Set backend before importing pyplot
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from matplotlib.figure import Figure
import matplotlib.patches as mpatches
import networkx as nx
import numpy as np


class PolishCitiesGraph:
    
    def __init__(self):
        self.graph = {
            "Glogow": {"Leszno": 45, "Wroclaw": 140},
            "Leszno": {"Glogow": 45, "Poznan": 90, "Kalisz": 130},
            "Poznan": {"Leszno": 90, "Bydgoszcz": 140, "Konin": 120},
            "Bydgoszcz": {"Poznan": 140, "Wloclawek": 110},
            "Wloclawek": {"Bydgoszcz": 110, "Plock": 55},
            "Konin": {"Poznan": 120, "Lodz": 120},
            "Kalisz": {"Leszno": 130, "Lodz": 120, "Czestochowa": 160},
            "Lodz": {"Konin": 120, "Kalisz": 120, "Warsaw": 150, "Radom": 165, "Krakow": 280},
            "Warsaw": {"Lodz": 150, "Plock": 130, "Radom": 105},
            "Radom": {"Warsaw": 105, "Kielce": 82, "Lodz": 165},
            "Kielce": {"Radom": 82, "Krakow": 120},
            "Krakow": {"Kielce": 120, "Katowice": 85, "Lodz": 280},
            "Katowice": {"Krakow": 85, "Czestochowa": 80, "Opole": 118},
            "Czestochowa": {"Kalisz": 160, "Katowice": 80},
            "Opole": {"Wroclaw": 100, "Katowice": 118},
            "Wroclaw": {"Glogow": 140, "Opole": 100},
            "Plock": {"Wloclawek": 55, "Warsaw": 130}
        }
        
        # Heuristic values (straight-line distance to Plock in km)
        self.heuristic = {
            "Glogow": 280, "Leszno": 250, "Poznan": 200, "Bydgoszcz": 100,
            "Wloclawek": 55, "Konin": 150, "Kalisz": 220, "Lodz": 130,
            "Warsaw": 100, "Radom": 180, "Kielce": 250, "Krakow": 320,
            "Katowice": 350, "Czestochowa": 280, "Opole": 320, "Wroclaw": 300,
            "Plock": 0
        }
        
        # Approximate positions for visualization
        self.positions = {
            "Glogow": (1.5, 5.5), "Leszno": (2.5, 5.0), "Wroclaw": (2.0, 4.0),
            "Poznan": (3.0, 6.0), "Bydgoszcz": (4.5, 7.0), "Wloclawek": (5.5, 6.5),
            "Plock": (6.0, 6.0), "Konin": (4.0, 5.0), "Kalisz": (3.5, 4.0),
            "Lodz": (5.0, 4.0), "Warsaw": (7.0, 5.0), "Radom": (7.0, 3.5),
            "Kielce": (6.5, 2.5), "Krakow": (5.5, 1.5), "Katowice": (4.5, 1.5),
            "Czestochowa": (4.0, 2.5), "Opole": (3.0, 2.5)
        }
        
        self.start = "Glogow"
        self.goal = "Plock"
    
    def get_neighbors(self, node):
        """Get neighbors of a node with their distances"""
        return self.graph.get(node, {})
    
    def get_cities(self):
        """Get list of all cities"""
        return sorted(self.graph.keys())
    
    def create_networkx_graph(self):
        """Create NetworkX graph for visualization"""
        G = nx.Graph()
        for node in self.graph:
            for neighbor, weight in self.graph[node].items():
                G.add_edge(node, neighbor, weight=weight)
        return G


class SearchAlgorithms:
    """
    Implementation of search algorithms: DFS, BFS, and A*
    Each algorithm returns a trace of its execution for visualization
    """
    
    @staticmethod
    def depth_first_search(graph_obj, start, goal, callback=None):
        """
        Depth-First Search using a stack (LIFO)
        
        Args:
            graph_obj: PolishCitiesGraph instance
            start: Starting city
            goal: Goal city
            callback: Function to call with trace updates
        
        Returns:
            tuple: (path, cost, nodes_expanded, trace, execution_time)
        """
        graph = graph_obj.graph
        start_time = time.time()
        
        open_stack = [(start, [start], 0)]
        closed_set = set()
        trace = []
        nodes_expanded = 0
        step = 0
        
        # Initial state
        trace.append({
            'step': step,
            'current': None,
            'path': [],
            'cost': 0,
            'open': [start],
            'closed': set(),
            'action': 'Initialize',
            'message': f'Starting DFS from {start} to {goal}'
        })
        
        if callback:
            callback(trace[-1])
        
        while open_stack:
            step += 1
            current_node, path, cost = open_stack.pop()
            
            # Record pop action
            trace.append({
                'step': step,
                'current': current_node,
                'path': path.copy(),
                'cost': cost,
                'open': [n[0] for n in open_stack],
                'closed': closed_set.copy(),
                'action': 'Pop from Stack',
                'message': f'Popped {current_node} (cost: {cost}km)'
            })
            
            if callback:
                callback(trace[-1])
                time.sleep(0.1)
            
            # Goal test
            if current_node == goal:
                execution_time = time.time() - start_time
                trace.append({
                    'step': step + 0.5,
                    'current': current_node,
                    'path': path.copy(),
                    'cost': cost,
                    'open': [n[0] for n in open_stack],
                    'closed': closed_set.copy(),
                    'action': 'GOAL FOUND!',
                    'message': f'Goal {goal} reached! Total cost: {cost}km'
                })
                
                if callback:
                    callback(trace[-1])
                
                return path, cost, nodes_expanded, trace, execution_time
            
            # Skip if already visited
            if current_node in closed_set:
                continue
            
            # Add to closed set
            closed_set.add(current_node)
            nodes_expanded += 1
            
            # Expand neighbors (reversed to maintain left-to-right ordering in stack)
            neighbors_added = []
            for neighbor, weight in reversed(list(graph[current_node].items())):
                if neighbor not in closed_set:
                    open_stack.append((neighbor, path + [neighbor], cost + weight))
                    neighbors_added.append(neighbor)
            
            if neighbors_added:
                trace.append({
                    'step': step + 0.5,
                    'current': current_node,
                    'path': path.copy(),
                    'cost': cost,
                    'open': [n[0] for n in open_stack],
                    'closed': closed_set.copy(),
                    'action': 'Expand',
                    'message': f'Added neighbors: {", ".join(neighbors_added)}'
                })
                
                if callback:
                    callback(trace[-1])
        
        # No path found
        execution_time = time.time() - start_time
        return None, None, nodes_expanded, trace, execution_time
    
    @staticmethod
    def breadth_first_search(graph_obj, start, goal, callback=None):
        """
        Breadth-First Search using a queue (FIFO)
        
        Args:
            graph_obj: PolishCitiesGraph instance
            start: Starting city
            goal: Goal city
            callback: Function to call with trace updates
        
        Returns:
            tuple: (path, cost, nodes_expanded, trace, execution_time)
        """
        graph = graph_obj.graph
        start_time = time.time()
        
        open_queue = deque([(start, [start], 0)])
        closed_set = set()
        trace = []
        nodes_expanded = 0
        step = 0
        
        # Initial state
        trace.append({
            'step': step,
            'current': None,
            'path': [],
            'cost': 0,
            'open': [start],
            'closed': set(),
            'action': 'Initialize',
            'message': f'Starting BFS from {start} to {goal}'
        })
        
        if callback:
            callback(trace[-1])
        
        while open_queue:
            step += 1
            current_node, path, cost = open_queue.popleft()
            
            # Record dequeue action
            trace.append({
                'step': step,
                'current': current_node,
                'path': path.copy(),
                'cost': cost,
                'open': [n[0] for n in open_queue],
                'closed': closed_set.copy(),
                'action': 'Dequeue',
                'message': f'Dequeued {current_node} (cost: {cost}km)'
            })
            
            if callback:
                callback(trace[-1])
                time.sleep(0.1)
            
            # Goal test
            if current_node == goal:
                execution_time = time.time() - start_time
                trace.append({
                    'step': step + 0.5,
                    'current': current_node,
                    'path': path.copy(),
                    'cost': cost,
                    'open': [n[0] for n in open_queue],
                    'closed': closed_set.copy(),
                    'action': 'GOAL FOUND!',
                    'message': f'Goal {goal} reached! Total cost: {cost}km'
                })
                
                if callback:
                    callback(trace[-1])
                
                return path, cost, nodes_expanded, trace, execution_time
            
            # Skip if already visited
            if current_node in closed_set:
                continue
            
            # Add to closed set
            closed_set.add(current_node)
            nodes_expanded += 1
            
            # Expand neighbors
            neighbors_added = []
            for neighbor, weight in sorted(graph[current_node].items()):
                if neighbor not in closed_set:
                    open_queue.append((neighbor, path + [neighbor], cost + weight))
                    neighbors_added.append(neighbor)
            
            if neighbors_added:
                trace.append({
                    'step': step + 0.5,
                    'current': current_node,
                    'path': path.copy(),
                    'cost': cost,
                    'open': [n[0] for n in open_queue],
                    'closed': closed_set.copy(),
                    'action': 'Expand',
                    'message': f'Added neighbors: {", ".join(neighbors_added)}'
                })
                
                if callback:
                    callback(trace[-1])
        
        # No path found
        execution_time = time.time() - start_time
        return None, None, nodes_expanded, trace, execution_time
    
    @staticmethod
    def a_star_search(graph_obj, start, goal, callback=None):
        graph = graph_obj.graph
        heuristic = graph_obj.heuristic
        start_time = time.time()
        
        open_pq = [(heuristic[start], 0, start, [start])]
        closed_set = set()
        trace = []
        nodes_expanded = 0
        step = 0
        
        # Initial state
        trace.append({
            'step': step,
            'current': None,
            'path': [],
            'cost': 0,
            'f': heuristic[start],
            'open': [(heuristic[start], start)],
            'closed': set(),
            'action': 'Initialize',
            'message': f'Starting A* from {start} to {goal}'
        })
        
        if callback:
            callback(trace[-1])
        
        while open_pq:
            step += 1
            f_score, g_score, current_node, path = heapq.heappop(open_pq)
            
            # Record pop action
            h_score = heuristic[current_node]
            trace.append({
                'step': step,
                'current': current_node,
                'path': path.copy(),
                'cost': g_score,
                'f': f_score,
                'open': [(f, n) for f, g, n, p in open_pq],
                'closed': closed_set.copy(),
                'action': 'Pop min f(n)',
                'message': f'Popped {current_node}: f={f_score} (g={g_score} + h={h_score})'
            })
            
            if callback:
                callback(trace[-1])
                time.sleep(0.1)
            
            # Goal test
            if current_node == goal:
                execution_time = time.time() - start_time
                trace.append({
                    'step': step + 0.5,
                    'current': current_node,
                    'path': path.copy(),
                    'cost': g_score,
                    'f': f_score,
                    'open': [(f, n) for f, g, n, p in open_pq],
                    'closed': closed_set.copy(),
                    'action': 'OPTIMAL GOAL!',
                    'message': f'Optimal path found! Total cost: {g_score}km'
                })
                
                if callback:
                    callback(trace[-1])
                
                return path, g_score, nodes_expanded, trace, execution_time
            
            # Skip if already visited
            if current_node in closed_set:
                continue
            
            # Add to closed set
            closed_set.add(current_node)
            nodes_expanded += 1
            
            # Expand neighbors
            neighbors_added = []
            for neighbor, weight in sorted(graph[current_node].items()):
                if neighbor not in closed_set:
                    new_g = g_score + weight
                    new_h = heuristic[neighbor]
                    new_f = new_g + new_h
                    heapq.heappush(open_pq, (new_f, new_g, neighbor, path + [neighbor]))
                    neighbors_added.append(f"{neighbor}(f={new_f})")
            
            if neighbors_added:
                trace.append({
                    'step': step + 0.5,
                    'current': current_node,
                    'path': path.copy(),
                    'cost': g_score,
                    'f': f_score,
                    'open': [(f, n) for f, g, n, p in open_pq],
                    'closed': closed_set.copy(),
                    'action': 'Expand',
                    'message': f'Added {len(neighbors_added)} neighbors'
                })
                
                if callback:
                    callback(trace[-1])
        
        # No path found
        execution_time = time.time() - start_time
        return None, None, nodes_expanded, trace, execution_time


class GraphVisualizationPanel:
    """
    Panel for displaying the graph and search progress
    """
    
    def __init__(self, parent, graph_obj):
        self.parent = parent
        self.graph_obj = graph_obj
        self.current_state = None
        
        # Create matplotlib figure
        self.figure = Figure(figsize=(8, 6), dpi=100)
        self.ax = self.figure.add_subplot(111)
        
        # Create canvas
        self.canvas = FigureCanvasTkAgg(self.figure, parent)
        self.canvas.get_tk_widget().pack(fill=tk.BOTH, expand=True)
        
        # Create NetworkX graph
        self.G = graph_obj.create_networkx_graph()
        
        # Initial draw
        self.draw_graph()
    
    def draw_graph(self, state=None):
        """Draw the graph with current state highlighting"""
        self.ax.clear()
        pos = self.graph_obj.positions
        
        if state is None:
            # Draw basic graph
            node_colors = ['#bdc3c7' for _ in self.G.nodes()]
            node_sizes = [1500 for _ in self.G.nodes()]
            
            nx.draw_networkx_edges(self.G, pos, edge_color='#95a5a6', 
                                 width=2, alpha=0.6, ax=self.ax)
            nx.draw_networkx_nodes(self.G, pos, node_color=node_colors,
                                 node_size=node_sizes, edgecolors='#34495e',
                                 linewidths=2, ax=self.ax)
            nx.draw_networkx_labels(self.G, pos, font_size=8, 
                                  font_weight='bold', ax=self.ax)
            
            edge_labels = nx.get_edge_attributes(self.G, 'weight')
            nx.draw_networkx_edge_labels(self.G, pos, edge_labels=edge_labels,
                                        font_size=7, ax=self.ax)
            
            self.ax.set_title('Polish Cities Road Network\nGlogow → Plock',
                            fontsize=12, fontweight='bold')
        else:
            # Draw graph with search state
            current = state.get('current')
            path = state.get('path', [])
            open_list = state.get('open', [])
            closed_set = state.get('closed', set())
            
            # Determine node colors
            node_colors = []
            node_sizes = []
            for node in self.G.nodes():
                if node == current:
                    node_colors.append('#f39c12')  # Orange - current
                    node_sizes.append(2000)
                elif node == self.graph_obj.start:
                    node_colors.append('#3498db')  # Blue - start
                    node_sizes.append(1800)
                elif node == self.graph_obj.goal:
                    node_colors.append('#e74c3c')  # Red - goal
                    node_sizes.append(1800)
                elif node in closed_set:
                    node_colors.append('#9b59b6')  # Purple - closed
                    node_sizes.append(1500)
                elif node in path:
                    node_colors.append('#2ecc71')  # Green - path
                    node_sizes.append(1600)
                elif isinstance(open_list[0], tuple) if open_list else False:
                    # A* case
                    if node in [n[1] for n in open_list]:
                        node_colors.append('#f1c40f')  # Yellow - open
                        node_sizes.append(1500)
                    else:
                        node_colors.append('#bdc3c7')  # Gray - unvisited
                        node_sizes.append(1300)
                else:
                    # DFS/BFS case
                    if node in open_list:
                        node_colors.append('#f1c40f')  # Yellow - open
                        node_sizes.append(1500)
                    else:
                        node_colors.append('#bdc3c7')  # Gray - unvisited
                        node_sizes.append(1300)
            
            # Draw edges
            nx.draw_networkx_edges(self.G, pos, edge_color='#95a5a6',
                                 width=2, alpha=0.4, ax=self.ax)
            
            # Draw path edges
            if len(path) > 1:
                path_edges = list(zip(path, path[1:]))
                nx.draw_networkx_edges(self.G, pos, edgelist=path_edges,
                                     edge_color='#27ae60', width=4,
                                     alpha=0.8, ax=self.ax)
            
            # Draw nodes
            nx.draw_networkx_nodes(self.G, pos, node_color=node_colors,
                                 node_size=node_sizes, edgecolors='#2c3e50',
                                 linewidths=2, ax=self.ax)
            
            # Draw labels
            nx.draw_networkx_labels(self.G, pos, font_size=7,
                                  font_weight='bold', ax=self.ax)
            
            # Draw edge weights
            edge_labels = nx.get_edge_attributes(self.G, 'weight')
            nx.draw_networkx_edge_labels(self.G, pos, edge_labels=edge_labels,
                                        font_size=6, ax=self.ax)
            
            # Add title with step info
            action = state.get('action', '')
            step = state.get('step', 0)
            cost = state.get('cost', 0)
            
            title = f"Step {int(step)}: {action}\n"
            if current:
                title += f"Current: {current} | Cost: {cost}km"
            
            self.ax.set_title(title, fontsize=11, fontweight='bold')
            
            # Add legend
            legend_elements = [
                mpatches.Patch(color='#3498db', label='Start'),
                mpatches.Patch(color='#e74c3c', label='Goal'),
                mpatches.Patch(color='#f39c12', label='Current'),
                mpatches.Patch(color='#f1c40f', label='Open'),
                mpatches.Patch(color='#9b59b6', label='Closed'),
                mpatches.Patch(color='#2ecc71', label='Path'),
            ]
            self.ax.legend(handles=legend_elements, loc='upper left',
                         fontsize=7, ncol=2)
        
        self.ax.axis('off')
        self.canvas.draw()
    
    def update_state(self, state):
        """Update visualization with new state"""
        self.current_state = state
        self.draw_graph(state)


class MainApplication:
    """
    Main GUI Application for Graph Search Visualization
    """
    
    def __init__(self, root):
        self.root = root
        self.root.title("Graph Search Algorithms - Robot Parcel Delivery System")
        self.root.geometry("1400x900")
        self.root.configure(bg='#ecf0f1')
        
        # Initialize graph
        self.graph_obj = PolishCitiesGraph()
        
        # Algorithm results storage
        self.results = {
            'dfs': None,
            'bfs': None,
            'astar': None
        }
        
        # Animation control
        self.is_animating = False
        self.animation_thread = None
        self.update_queue = queue.Queue()
        
        # Setup UI
        self.setup_ui()
        
        # Start queue processor
        self.process_queue()
    
    def setup_ui(self):
        """Setup the user interface"""
        
        # Title Frame
        title_frame = tk.Frame(self.root, bg='#2c3e50', height=80)
        title_frame.pack(fill=tk.X, padx=0, pady=0)
        title_frame.pack_propagate(False)
        
        title_label = tk.Label(
            title_frame,
            text="Graph Search Algorithms Visualization",
            font=('Arial', 24, 'bold'),
            bg='#2c3e50',
            fg='white'
        )
        title_label.pack(pady=10)
        
        subtitle_label = tk.Label(
            title_frame,
            text="Robot Parcel Delivery: Glogow → Plock",
            font=('Arial', 14),
            bg='#2c3e50',
            fg='#ecf0f1'
        )
        subtitle_label.pack()
        
        # Main Content Frame
        content_frame = tk.Frame(self.root, bg='#ecf0f1')
        content_frame.pack(fill=tk.BOTH, expand=True, padx=10, pady=10)
        
        # Left Panel - Graph Visualization
        left_panel = tk.LabelFrame(
            content_frame,
            text="Graph Visualization",
            font=('Arial', 12, 'bold'),
            bg='white',
            fg='#2c3e50',
            padx=10,
            pady=10
        )
        left_panel.grid(row=0, column=0, sticky='nsew', padx=5, pady=5)
        
        self.graph_panel = GraphVisualizationPanel(left_panel, self.graph_obj)
        
        # Right Panel
        right_panel = tk.Frame(content_frame, bg='#ecf0f1')
        right_panel.grid(row=0, column=1, sticky='nsew', padx=5, pady=5)
        
        # Control Panel
        control_frame = tk.LabelFrame(
            right_panel,
            text="Algorithm Controls",
            font=('Arial', 12, 'bold'),
            bg='white',
            fg='#2c3e50',
            padx=15,
            pady=15
        )
        control_frame.pack(fill=tk.X, pady=(0, 10))
        
        # Algorithm selection
        tk.Label(
            control_frame,
            text="Select Algorithm:",
            font=('Arial', 10, 'bold'),
            bg='white'
        ).grid(row=0, column=0, sticky='w', pady=5)
        
        self.algorithm_var = tk.StringVar(value='astar')
        
        algorithms = [
            ('Depth-First Search (DFS)', 'dfs'),
            ('Breadth-First Search (BFS)', 'bfs'),
            ('A* Search (Optimal)', 'astar')
        ]
        
        for i, (text, value) in enumerate(algorithms):
            tk.Radiobutton(
                control_frame,
                text=text,
                variable=self.algorithm_var,
                value=value,
                font=('Arial', 10),
                bg='white',
                activebackground='white'
            ).grid(row=i+1, column=0, sticky='w', padx=20)
        
        # Animation speed
        tk.Label(
            control_frame,
            text="Animation Speed:",
            font=('Arial', 10, 'bold'),
            bg='white'
        ).grid(row=4, column=0, sticky='w', pady=(10, 5))
        
        self.speed_var = tk.DoubleVar(value=0.5)
        speed_scale = tk.Scale(
            control_frame,
            from_=0.1,
            to=2.0,
            resolution=0.1,
            orient=tk.HORIZONTAL,
            variable=self.speed_var,
            bg='white',
            highlightthickness=0
        )
        speed_scale.grid(row=5, column=0, sticky='ew', padx=20)
        
        # Buttons
        button_frame = tk.Frame(control_frame, bg='white')
        button_frame.grid(row=6, column=0, pady=15)
        
        self.run_button = tk.Button(
            button_frame,
            text="Run Algorithm",
            command=self.run_algorithm,
            font=('Arial', 11, 'bold'),
            bg='#27ae60',
            fg='white',
            activebackground='#229954',
            activeforeground='white',
            padx=20,
            pady=10,
            cursor='hand2',
            relief=tk.RAISED,
            bd=3
        )
        self.run_button.pack(side=tk.LEFT, padx=5)
        
        self.stop_button = tk.Button(
            button_frame,
            text="Stop",
            command=self.stop_animation,
            font=('Arial', 11, 'bold'),
            bg='#e74c3c',
            fg='white',
            activebackground='#c0392b',
            activeforeground='white',
            padx=20,
            pady=10,
            cursor='hand2',
            relief=tk.RAISED,
            bd=3,
            state=tk.DISABLED
        )
        self.stop_button.pack(side=tk.LEFT, padx=5)
        
        tk.Button(
            button_frame,
            text="Reset",
            command=self.reset_visualization,
            font=('Arial', 11, 'bold'),
            bg='#3498db',
            fg='white',
            activebackground='#2980b9',
            activeforeground='white',
            padx=20,
            pady=10,
            cursor='hand2',
            relief=tk.RAISED,
            bd=3
        ).pack(side=tk.LEFT, padx=5)
        
        # Compare All button
        tk.Button(
            control_frame,
            text="Compare All Algorithms",
            command=self.compare_all,
            font=('Arial', 11, 'bold'),
            bg='#9b59b6',
            fg='white',
            activebackground='#8e44ad',
            activeforeground='white',
            padx=20,
            pady=10,
            cursor='hand2',
            relief=tk.RAISED,
            bd=3
        ).grid(row=7, column=0, pady=10)
        
        # Info Panel
        info_frame = tk.LabelFrame(
            right_panel,
            text="Algorithm Information",
            font=('Arial', 12, 'bold'),
            bg='white',
            fg='#2c3e50',
            padx=10,
            pady=10
        )
        info_frame.pack(fill=tk.BOTH, expand=True)
        
        # Create scrolled text for info
        self.info_text = scrolledtext.ScrolledText(
            info_frame,
            wrap=tk.WORD,
            font=('Courier', 9),
            bg='#f8f9fa',
            fg='#2c3e50',
            height=20
        )
        self.info_text.pack(fill=tk.BOTH, expand=True)
        
        # Add initial info
        self.display_initial_info()
        
        # Configure grid weights
        content_frame.grid_columnconfigure(0, weight=2)
        content_frame.grid_columnconfigure(1, weight=1)
        content_frame.grid_rowconfigure(0, weight=1)
        
        # Status bar
        status_frame = tk.Frame(self.root, bg='#34495e', height=30)
        status_frame.pack(fill=tk.X, side=tk.BOTTOM)
        status_frame.pack_propagate(False)
        
        self.status_label = tk.Label(
            status_frame,
            text="Ready",
            font=('Arial', 9),
            bg='#34495e',
            fg='white',
            anchor='w'
        )
        self.status_label.pack(fill=tk.X, padx=10)
    
    def display_initial_info(self):
        """Display initial information about the system"""
        info = f"""
════════════════════════════════════════════════════════
        GRAPH SEARCH VISUALIZATION SYSTEM              
════════════════════════════════════════════════════════

PROBLEM DESCRIPTION:
   A robot needs to deliver a parcel from Glogow to Plock
   using the shortest path through Polish cities.

STATE SPACE:
   • Total Cities: {len(self.graph_obj.graph)}
   • Total Roads: {sum(len(v) for v in self.graph_obj.graph.values()) // 2}
   • Start City: {self.graph_obj.start}
   • Goal City: {self.graph_obj.goal}

ALGORITHMS AVAILABLE:

1. DEPTH-FIRST SEARCH (DFS)
   • Uses: Stack (LIFO)
   • Complete: No
   • Optimal: No
   • Memory: O(bm) - Low

2. BREADTH-FIRST SEARCH (BFS)
   • Uses: Queue (FIFO)
   • Complete: Yes
   • Optimal: For unweighted graphs only
   • Memory: O(b^d) - High

3. A* SEARCH
   • Uses: Priority Queue (f = g + h)
   • Complete: Yes
   • Optimal: Yes (with admissible heuristic)
   • Memory: O(b^d) - High

INSTRUCTIONS:
   1. Select an algorithm from the controls
   2. Adjust animation speed if desired
   3. Click "Run Algorithm" to start visualization
   4. Watch the search progress in real-time
   5. Use "Compare All" to see performance metrics

════════════════════════════════════════════════════════
"""
        self.info_text.insert('1.0', info)
        self.info_text.config(state=tk.DISABLED)
    
    def update_info(self, state):
        """Update info panel with current state"""
        self.info_text.config(state=tk.NORMAL)
        self.info_text.delete('1.0', tk.END)
        
        current = state.get('current', 'N/A')
        step = int(state.get('step', 0))
        action = state.get('action', '')
        message = state.get('message', '')
        path = state.get('path', [])
        cost = state.get('cost', 0)
        open_list = state.get('open', [])
        closed_set = state.get('closed', set())
        
        # Format open list
        if open_list and isinstance(open_list[0], tuple):
            # A* case
            open_display = [f"{n[1]}(f={n[0]})" for n in open_list[:8]]
        else:
            open_display = list(open_list[:8]) if open_list else []
        
        open_str = " -> ".join(map(str, open_display)) if open_display else "(empty)"
        if len(open_list) > 8:
            open_str += f" ... (+{len(open_list)-8} more)"
        
        closed_str = ", ".join(sorted(list(closed_set)[:12])) if closed_set else "(empty)"
        if len(closed_set) > 12:
            closed_str += f" ... (+{len(closed_set)-12} more)"
        
        path_str = " -> ".join(path) if path else "(empty)"
        
        info = f"""
════════════════════════════════════════════════════════
              ALGORITHM EXECUTION STATE                
════════════════════════════════════════════════════════

CURRENT STATUS:
   Step: {step}
   Action: {action}
   Message: {message}

CURRENT NODE:
   {current}

PATH SO FAR:
   {path_str}
   
COST: {cost} km

OPEN CONTAINER:
   {open_str}

CLOSED SET:
   {closed_str}

════════════════════════════════════════════════════════
"""
        
        self.info_text.insert('1.0', info)
        self.info_text.config(state=tk.DISABLED)
    
    def run_algorithm(self):
        """Run the selected algorithm"""
        if self.is_animating:
            messagebox.showwarning("Animation Running", 
                                 "Please stop the current animation first.")
            return
        
        algorithm = self.algorithm_var.get()
        self.is_animating = True
        self.run_button.config(state=tk.DISABLED)
        self.stop_button.config(state=tk.NORMAL)
        self.status_label.config(text=f"Running {algorithm.upper()}...")
        
        # Run algorithm in separate thread
        self.animation_thread = threading.Thread(
            target=self._run_algorithm_thread,
            args=(algorithm,),
            daemon=True
        )
        self.animation_thread.start()
    
    def _run_algorithm_thread(self, algorithm):
        """Thread function to run algorithm"""
        
        def callback(state):
            """Callback to update visualization"""
            if not self.is_animating:
                return
            
            # Add state to queue
            self.update_queue.put(('state', state))
            
            # Sleep based on animation speed
            delay = 1.0 / self.speed_var.get()
            time.sleep(delay)
        
        try:
            # Run the appropriate algorithm
            if algorithm == 'dfs':
                result = SearchAlgorithms.depth_first_search(
                    self.graph_obj,
                    self.graph_obj.start,
                    self.graph_obj.goal,
                    callback
                )
            elif algorithm == 'bfs':
                result = SearchAlgorithms.breadth_first_search(
                    self.graph_obj,
                    self.graph_obj.start,
                    self.graph_obj.goal,
                    callback
                )
            else:  # astar
                result = SearchAlgorithms.a_star_search(
                    self.graph_obj,
                    self.graph_obj.start,
                    self.graph_obj.goal,
                    callback
                )
            
            # Store result
            path, cost, nodes_expanded, trace, execution_time = result
            self.results[algorithm] = {
                'path': path,
                'cost': cost,
                'nodes_expanded': nodes_expanded,
                'trace': trace,
                'execution_time': execution_time,
                'path_length': len(path) if path else 0
            }
            
            # Signal completion
            self.update_queue.put(('complete', algorithm, result))
            
        except Exception as e:
            self.update_queue.put(('error', str(e)))
    
    def stop_animation(self):
        """Stop the current animation"""
        self.is_animating = False
        self.run_button.config(state=tk.NORMAL)
        self.stop_button.config(state=tk.DISABLED)
        self.status_label.config(text="Animation stopped")
    
    def reset_visualization(self):
        """Reset the visualization"""
        self.is_animating = False
        self.run_button.config(state=tk.NORMAL)
        self.stop_button.config(state=tk.DISABLED)
        self.graph_panel.draw_graph()
        self.display_initial_info()
        self.status_label.config(text="Ready")
    
    def process_queue(self):
        """Process updates from the algorithm thread"""
        try:
            while True:
                item = self.update_queue.get_nowait()
                
                if item[0] == 'state':
                    state = item[1]
                    self.graph_panel.update_state(state)
                    self.update_info(state)
                
                elif item[0] == 'complete':
                    algorithm = item[1]
                    result = item[2]
                    path, cost, nodes_expanded, trace, execution_time = result
                    
                    self.is_animating = False
                    self.run_button.config(state=tk.NORMAL)
                    self.stop_button.config(state=tk.DISABLED)
                    
                    # Show result
                    if path:
                        message = f"""
{algorithm.upper()} COMPLETED!

Path Found: {' -> '.join(path)}
Total Cost: {cost} km
Path Length: {len(path)} cities
Nodes Expanded: {nodes_expanded}
Execution Time: {execution_time*1000:.2f} ms
"""
                        messagebox.showinfo(f"{algorithm.upper()} Complete", message)
                        self.status_label.config(text=f"{algorithm.upper()} completed - Cost: {cost}km")
                    else:
                        messagebox.showerror("No Path", "No path found!")
                        self.status_label.config(text="No path found")
                
                elif item[0] == 'error':
                    error = item[1]
                    self.is_animating = False
                    self.run_button.config(state=tk.NORMAL)
                    self.stop_button.config(state=tk.DISABLED)
                    messagebox.showerror("Error", f"An error occurred: {error}")
                    self.status_label.config(text="Error occurred")
        
        except queue.Empty:
            pass
        
        # Schedule next check
        self.root.after(50, self.process_queue)
    
    def compare_all(self):
        """Run all algorithms and show comparison"""
        if self.is_animating:
            messagebox.showwarning("Animation Running",
                                 "Please stop the current animation first.")
            return
        
        # Show progress window
        progress_window = tk.Toplevel(self.root)
        progress_window.title("Running Comparison...")
        progress_window.geometry("400x150")
        progress_window.transient(self.root)
        progress_window.grab_set()
        
        tk.Label(
            progress_window,
            text="Running all algorithms...",
            font=('Arial', 12, 'bold')
        ).pack(pady=20)
        
        progress_label = tk.Label(
            progress_window,
            text="",
            font=('Arial', 10)
        )
        progress_label.pack()
        
        progress_bar = ttk.Progressbar(
            progress_window,
            mode='determinate',
            length=300
        )
        progress_bar.pack(pady=20)
        
        def run_comparison():
            algorithms = [('dfs', 'DFS'), ('bfs', 'BFS'), ('astar', 'A*')]
            
            for i, (algo, name) in enumerate(algorithms):
                progress_label.config(text=f"Running {name}...")
                progress_bar['value'] = (i / len(algorithms)) * 100
                progress_window.update()
                
                # Run algorithm without animation
                if algo == 'dfs':
                    result = SearchAlgorithms.depth_first_search(
                        self.graph_obj,
                        self.graph_obj.start,
                        self.graph_obj.goal
                    )
                elif algo == 'bfs':
                    result = SearchAlgorithms.breadth_first_search(
                        self.graph_obj,
                        self.graph_obj.start,
                        self.graph_obj.goal
                    )
                else:
                    result = SearchAlgorithms.a_star_search(
                        self.graph_obj,
                        self.graph_obj.start,
                        self.graph_obj.goal
                    )
                
                path, cost, nodes_expanded, trace, execution_time = result
                self.results[algo] = {
                    'path': path,
                    'cost': cost,
                    'nodes_expanded': nodes_expanded,
                    'trace': trace,
                    'execution_time': execution_time,
                    'path_length': len(path) if path else 0
                }
            
            progress_bar['value'] = 100
            progress_label.config(text="Complete!")
            progress_window.update()
            time.sleep(0.5)
            progress_window.destroy()
            
            # Show comparison window
            self.show_comparison_window()
        
        # Run in thread
        threading.Thread(target=run_comparison, daemon=True).start()
    
    def show_comparison_window(self):
        """Show comparison results window"""
        comp_window = tk.Toplevel(self.root)
        comp_window.title("Algorithm Comparison")
        comp_window.geometry("900x700")
        
        # Title
        tk.Label(
            comp_window,
            text="Algorithm Performance Comparison",
            font=('Arial', 16, 'bold'),
            bg='#2c3e50',
            fg='white',
            pady=15
        ).pack(fill=tk.X)
        
        # Create notebook for tabs
        notebook = ttk.Notebook(comp_window)
        notebook.pack(fill=tk.BOTH, expand=True, padx=10, pady=10)
        
        # Tab 1: Comparison Table
        table_frame = tk.Frame(notebook, bg='white')
        notebook.add(table_frame, text="Comparison Table")
        
        # Create table
        columns = ('Metric', 'DFS', 'BFS', 'A*')
        tree = ttk.Treeview(table_frame, columns=columns, show='headings', height=15)
        
        for col in columns:
            tree.heading(col, text=col)
            tree.column(col, width=200, anchor='center')
        
        # Add data
        metrics_data = [
            ('Path Cost (km)',
             self.results['dfs']['cost'],
             self.results['bfs']['cost'],
             self.results['astar']['cost']),
            ('Path Length (cities)',
             self.results['dfs']['path_length'],
             self.results['bfs']['path_length'],
             self.results['astar']['path_length']),
            ('Nodes Expanded',
             self.results['dfs']['nodes_expanded'],
             self.results['bfs']['nodes_expanded'],
             self.results['astar']['nodes_expanded']),
            ('Execution Time (ms)',
             f"{self.results['dfs']['execution_time']*1000:.2f}",
             f"{self.results['bfs']['execution_time']*1000:.2f}",
             f"{self.results['astar']['execution_time']*1000:.2f}"),
        ]
        
        for data in metrics_data:
            tree.insert('', tk.END, values=data)
        
        tree.pack(fill=tk.BOTH, expand=True, padx=20, pady=20)
        
        # Tab 2: Charts
        chart_frame = tk.Frame(notebook, bg='white')
        notebook.add(chart_frame, text="Performance Charts")
        
        # Create comparison charts
        fig = Figure(figsize=(8, 6), dpi=100)
        
        # Create subplots
        gs = fig.add_gridspec(2, 2, hspace=0.3, wspace=0.3)
        
        algorithms = ['DFS', 'BFS', 'A*']
        colors = ['#e74c3c', '#3498db', '#2ecc71']
        
        # Chart 1: Path Cost
        ax1 = fig.add_subplot(gs[0, 0])
        costs = [self.results[a]['cost'] for a in ['dfs', 'bfs', 'astar']]
        bars = ax1.bar(algorithms, costs, color=colors)
        ax1.set_ylabel('Cost (km)')
        ax1.set_title('Path Cost Comparison')
        for bar, cost in zip(bars, costs):
            height = bar.get_height()
            ax1.text(bar.get_x() + bar.get_width()/2., height,
                    f'{int(cost)} km', ha='center', va='bottom', fontsize=9)
        
        # Highlight optimal
        min_cost = min(costs)
        for bar, cost in zip(bars, costs):
            if cost == min_cost:
                bar.set_edgecolor('gold')
                bar.set_linewidth(4)
        
        # Chart 2: Nodes Expanded
        ax2 = fig.add_subplot(gs[0, 1])
        expanded = [self.results[a]['nodes_expanded'] for a in ['dfs', 'bfs', 'astar']]
        bars = ax2.bar(algorithms, expanded, color=colors)
        ax2.set_ylabel('Nodes')
        ax2.set_title('Nodes Expanded')
        for bar, exp in zip(bars, expanded):
            height = bar.get_height()
            ax2.text(bar.get_x() + bar.get_width()/2., height,
                    f'{int(exp)}', ha='center', va='bottom', fontsize=9)
        
        # Chart 3: Path Length
        ax3 = fig.add_subplot(gs[1, 0])
        lengths = [self.results[a]['path_length'] for a in ['dfs', 'bfs', 'astar']]
        bars = ax3.bar(algorithms, lengths, color=colors)
        ax3.set_ylabel('Cities')
        ax3.set_title('Path Length')
        for bar, length in zip(bars, lengths):
            height = bar.get_height()
            ax3.text(bar.get_x() + bar.get_width()/2., height,
                    f'{int(length)}', ha='center', va='bottom', fontsize=9)
        
        # Chart 4: Execution Time
        ax4 = fig.add_subplot(gs[1, 1])
        times = [self.results[a]['execution_time']*1000 for a in ['dfs', 'bfs', 'astar']]
        bars = ax4.bar(algorithms, times, color=colors)
        ax4.set_ylabel('Time (ms)')
        ax4.set_title('Execution Time')
        for bar, t in zip(bars, times):
            height = bar.get_height()
            ax4.text(bar.get_x() + bar.get_width()/2., height,
                    f'{t:.2f} ms', ha='center', va='bottom', fontsize=8)
        
        canvas = FigureCanvasTkAgg(fig, chart_frame)
        canvas.draw()
        canvas.get_tk_widget().pack(fill=tk.BOTH, expand=True, padx=10, pady=10)
        
        # Tab 3: Analysis
        analysis_frame = tk.Frame(notebook, bg='white')
        notebook.add(analysis_frame, text="Analysis")
        
        analysis_text = scrolledtext.ScrolledText(
            analysis_frame,
            wrap=tk.WORD,
            font=('Courier', 10),
            bg='#f8f9fa'
        )
        analysis_text.pack(fill=tk.BOTH, expand=True, padx=20, pady=20)
        
        analysis_content = f"""
═══════════════════════════════════════════════════════════════════
              ALGORITHM COMPARISON & ANALYSIS                      
═══════════════════════════════════════════════════════════════════

RESULTS SUMMARY:

DFS Path:  {' -> '.join(self.results['dfs']['path'])}
   Cost: {self.results['dfs']['cost']} km | Length: {self.results['dfs']['path_length']} cities

BFS Path:  {' -> '.join(self.results['bfs']['path'])}
   Cost: {self.results['bfs']['cost']} km | Length: {self.results['bfs']['path_length']} cities

A* Path:   {' -> '.join(self.results['astar']['path'])}
   Cost: {self.results['astar']['cost']} km | Length: {self.results['astar']['path_length']} cities


═══════════════════════════════════════════════════════════════════

KEY FINDINGS:

1. OPTIMALITY:
   • A* found the optimal path with cost {self.results['astar']['cost']}km
   • DFS cost: {self.results['dfs']['cost']}km ({self.results['dfs']['cost'] - self.results['astar']['cost']}km more than optimal)
   • BFS cost: {self.results['bfs']['cost']}km ({self.results['bfs']['cost'] - self.results['astar']['cost']}km more than optimal)

2. EFFICIENCY:
   • DFS expanded {self.results['dfs']['nodes_expanded']} nodes
   • BFS expanded {self.results['bfs']['nodes_expanded']} nodes
   • A* expanded {self.results['astar']['nodes_expanded']} nodes

3. EXECUTION TIME:
   • DFS: {self.results['dfs']['execution_time']*1000:.2f}ms
   • BFS: {self.results['bfs']['execution_time']*1000:.2f}ms
   • A*: {self.results['astar']['execution_time']*1000:.2f}ms


═══════════════════════════════════════════════════════════════════

ADVANTAGES & DISADVANTAGES:

DEPTH-FIRST SEARCH (DFS)
─────────────────────────────────────────────────────────────────
ADVANTAGES:
   • Memory efficient: O(bm)
   • Simple implementation using stack
   • Good for finding any solution quickly

DISADVANTAGES:
   • NOT optimal (found {self.results['dfs']['cost']}km vs optimal {self.results['astar']['cost']}km)
   • May explore very deep paths unnecessarily
   • Solution quality depends on edge ordering

BREADTH-FIRST SEARCH (BFS)
─────────────────────────────────────────────────────────────────
ADVANTAGES:
   • Complete: Always finds solution
   • Optimal for unweighted graphs
   • Systematic level-by-level exploration

DISADVANTAGES:
   • NOT optimal for weighted graphs ({self.results['bfs']['cost']}km vs {self.results['astar']['cost']}km)
   • High memory usage: O(b^d)
   • Explores many unnecessary nodes

A* SEARCH
─────────────────────────────────────────────────────────────────
ADVANTAGES:
   • OPTIMAL: Guaranteed shortest path!
   • Complete: Will find solution if exists
   • Efficient: Uses heuristic to guide search
   • Best for weighted graphs like road networks

DISADVANTAGES:
   • Requires good heuristic design
   • Memory intensive: O(b^d)
   • Computational overhead for f(n) calculation


═══════════════════════════════════════════════════════════════════

CONCLUSIONS:

1. For robot delivery with weighted graphs: A* is the BEST choice
2. A* provides optimal solution with reasonable efficiency
3. DFS is only suitable when any solution is acceptable
4. BFS is optimal only for unweighted graphs (hop count)
5. Heuristic quality directly impacts A* performance

The optimal delivery route from Glogow to Plock is:
{' -> '.join(self.results['astar']['path'])}
Total distance: {self.results['astar']['cost']} km

═══════════════════════════════════════════════════════════════════
"""
        
        analysis_text.insert('1.0', analysis_content)
        analysis_text.config(state=tk.DISABLED)


def main():
    """Main entry point"""
    root = tk.Tk()
    app = MainApplication(root)
    root.mainloop()


if __name__ == "__main__":
    main()